In [ ]:
# TODO
# GPU + docker

In [1]:
import import_ipynb
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import Adam, SGD

from sampler import *
from data_modules import *
from callbacks import *

In [2]:
def generate_data(n, output_img):
    local = sample_no_signalling(n, True, output_img)
    non_local = sample_no_signalling(n, False, output_img)
    data = local + non_local
    targets = [0] * n + [1] * n
    return data, targets

In [3]:
data, targets = generate_data(1000, False)

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 5, 3)
        self.conv2 = nn.Conv2d(5, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1600, 100)
        self.fc2 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def features(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.sigmoid(x)
    
class MLP(nn.Module):
    def __init__(self, input_len, hidden_len):
        super().__init__()
        self.fc1 = nn.Linear(input_len, hidden_len)
        self.fc2 = nn.Linear(hidden_len, hidden_len)
        self.fc3 = nn.Linear(hidden_len, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.float()
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)

In [9]:
from torchmetrics import Accuracy
from torch.nn import BCEWithLogitsLoss

class Classifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = torch.nn.BCELoss()
        self.acc = Accuracy()
        
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        return SGD(self.parameters(), lr=1e-3)
        
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.model(x)
        y = y.float().unsqueeze(1)
        loss = self.loss(y_hat, y)
        #self.log('train_loss', loss)
        #self.log('train acc', self.acc(torch.round(y_hat), y.int()))
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.model(x)
        y = y.float().unsqueeze(1)
        val_loss = self.loss(y_hat, y)
        #self.log('val loss', val_loss)
        #self.log('val acc', self.acc(torch.round(y_hat), y.int()))

        return val_loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.model(x)
        y = y.float().unsqueeze(1)
        loss = self.loss(y_hat, y)
        output = dict({
            'test_loss': loss,
            'test_acc': self.acc(torch.round(y_hat), y.int()),
        })
        return output

In [12]:
!pip install -U pytorch-lightning

     |████████████████████████████████| 525 kB 1.0 MB/s eta 0:00:01
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.5.4
    Uninstalling pytorch-lightning-1.5.4:
      Successfully uninstalled pytorch-lightning-1.5.4


You should consider upgrading via the '/home/johny/.virtualenvs/pytorch/bin/python -m pip install --upgrade pip' command.


In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
 #   transforms.ToTensor(),
#    transforms.Resize(100)
])
dm = MyDataModule(data, targets, transform, 16)
clf = Classifier(MLP(16, 100))
trainer = pl.Trainer(max_epochs=500,
                     gpus=1, callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger,
                    log_every_n_steps=500)
trainer.fit(clf, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | MLP      | 11.9 K
1 | loss  | BCELoss  | 0     
2 | acc   | Accuracy | 0     
-----------------------------------
11.9 K    Trainable params
0         Non-trainable params
11.9 K    Total params
0.048     Total estimated model params size (MB)


Epoch 0:  74%|█████████▋   | 23/31 [00:00<00:00, 27.78it/s, loss=0.698, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█████████████| 31/31 [00:01<00:00, 21.29it/s, loss=0.698, v_num=6]
                                                                                

Epoch 1:  77%|██████████   | 24/31 [00:00<00:00, 27.30it/s, loss=0.698, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 2:  77%|██████████   | 24/31 [00:00<00:00, 29.64it/s, loss=0.698, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 3:  77%|██████████   | 24/31 [00:00<00:00, 26.63it/s, loss=0.698, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 4:  77%|██████████   | 24/31 [00:01<00:00, 23.84it/s, loss=0.698, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 5:  77%|██████████   | 24/31 [00:00<00:00, 28.56it/s, loss=0.697, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 6:  77%|██████████   | 24/31 [00:00<00:00, 32.13it/s, loss=0.697, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 7:  77%|██████████   | 24/31 [00:00<00:00, 28.79it/s, loss=0.697, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 8:  77%|██████████   | 24/31 [00:00<00:00, 30.30it/s, loss=0.697, v_num=6]
Validating: 0it [00:00, ?it/s]
Epoch 9:  77%|██████████   | 24/31 [00:00<00:00, 31.10it/s, loss=0.697, v_num=6]
Validating: 0it [00:00,

In [8]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
 #   transforms.ToTensor(),
#    transforms.Resize(100)
])
dm = MyDataModule(data, targets, transform, 16)
clf = Classifier(MLP(16, 100))
trainer = pl.Trainer(max_epochs=500,
                     gpus=1, callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger)
trainer.fit(clf, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | MLP      | 11.9 K
1 | loss  | BCELoss  | 0     
2 | acc   | Accuracy | 0     
-----------------------------------
11.9 K    Trainable params
0         Non-trainable params
11.9 K    Total params
0.048     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

train_loss│train acc│val loss│val acc
─────────────────────────────────────
0.6881370544433594│0.5600000023841858│0.6915006637573242│0.5093333125114441


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
next(iter(dm.train_dataset))

In [ ]:
data[2]

In [ ]:
dm.setup(None)

In [ ]:
dm = MyDataModule(data, targets, None, 16)

In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(100)
])
dm = MNISTDataModule()
clf2 = Classifier() 
trainer = pl.Trainer(max_epochs=25,
                     gpus=1, callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger)
trainer.fit(clf2, dm)

In [ ]:
a = clf.model.features(next(iter(dm.train_dataloader()))[0])

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(16):
    ax = plt.subplot(4,4,i+1)
    ax.imshow(a[0,i,:,:].detach().numpy())

In [ ]:
features = clf2.model.features(next(iter(dm.train_dataloader()))[0])

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(16):
    ax = plt.subplot(4,4,i+1)
    ax.imshow(features[2,i,:,:].detach().numpy(), cmap='gray')